# Ensemble e Florestas Aleatórias
Agregar as previsões de um conjunto de previsores em cima de um conjunto de dados as vezes é melhor do que a previsão individual de um previsor, issa tecnica é chamada de Ensemble Learning.

## Hard Voting
Reunir previsões de um conjunto de classificadores obtendo a classe que teve a maioria dos votos, pode gerar um classificador melhor.

In [1]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_moons

X, y = make_moons(n_samples=500, noise=0.30, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

log_clf = LogisticRegression()
rnd_clf = RandomForestClassifier()
svm_clf = SVC()

voting_clf = VotingClassifier(
    estimators=[('lr', log_clf), ('rf', rnd_clf), ('svc', svm_clf)],
    voting='hard')
voting_clf.fit(X_train, y_train)

VotingClassifier(estimators=[('lr', LogisticRegression()),
                             ('rf', RandomForestClassifier()), ('svc', SVC())])

In [2]:
#Verificando a acurácia de cada classificador
from sklearn.metrics import accuracy_score

for clf in (log_clf, rnd_clf, svm_clf, voting_clf):
    clf.fit(X_train,y_train)
    y_pred = clf.predict(X_test)
    print(clf.__class__.__name__, accuracy_score(y_test, y_pred))
    

LogisticRegression 0.864
RandomForestClassifier 0.896
SVC 0.896
VotingClassifier 0.904


O classificador de votação supera todos os outros mesmo que por pouco.

## Soft Voting
Se todos os classificadores usados terem o metodo predict_proba() então pode se usar o softvoting apenas alterando o parametro voting de 'hard' para 'soft', geralmente o resultado é melhor.
No exemplo acima foi usado SVC porém ele não tem o metodo mas alterando o parametro propability para 'True' fara com que ele tenha um predict_proba().

## Bagging e Pastin
Permitem que as instancias de treinamento sejam amostradas várias vezes por meio de múltiplos previsores mas só bagging permite que sejam amostrados pelo mesmo previsor, os previsores podem ser treinados em paralelo, essa é uma das razões para que bagging e pastin sejam métodos tão populares pois escalam muito bem.

Para usar bagging e pastin precisa da classe BaggingClassifier (ou BaggingRegressor para regressão). O código a seguir treina um conjunto de 500 classificadores de árvore de decisão, cada um treinado em 100 instancias de treinamento amostradas aleatoriamente, se quiser utilizar pasting precisa passar o parametro 'bootstrap' para False, 'n_jobs' é a quantidade de núcleos de CPU que devem ser utilizados, -1 significa que usa todos os disponíveis.

In [3]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

bag_clf = BaggingClassifier(
    DecisionTreeClassifier(), n_estimators=500,
    max_samples=100, bootstrap=True, n_jobs=-1)
bag_clf.fit(X_train, y_train)
y_pred = bag_clf.predict(X_test)


In [4]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, y_pred))


0.92


In [5]:
tree_clf = DecisionTreeClassifier(random_state=42)
tree_clf.fit(X_train, y_train)
y_pred_tree = tree_clf.predict(X_test)
print(accuracy_score(y_test, y_pred_tree))


0.856


## Florestas Aleatórias
É um ensemble de Árvores de Decisão.

In [8]:
from sklearn.ensemble import RandomForestClassifier

rnd_clf = RandomForestClassifier(n_estimators=500, max_leaf_nodes=16, n_jobs=-1)
rnd_clf.fit(X_train,y_train)

y_pred_ref = rnd_clf.predict(X_test)
print(accuracy_score(y_test, y_pred))

0.92


### Importancia das características
Mede analisando a importancia de uma característica analisando o quanto os nós da árvores que a utilizam reduzem, na média, a impureza. Pode se acessar o resultado através da variável feature_importances_.


In [11]:
from sklearn.datasets import load_iris

iris = load_iris()
rnd_clf = RandomForestClassifier(n_estimators=500, n_jobs=-1)
rnd_clf.fit(iris["data"], iris["target"])
for name, score in zip(iris["feature_names"], rnd_clf.feature_importances_):
    print(name,score)

sepal length (cm) 0.1076344903010838
sepal width (cm) 0.021843481009119422
petal length (cm) 0.4139838411809752
petal width (cm) 0.45653818750882164


## Boosting
Treinar previsores sequencialmente assim tentando melhorar a cada "etapa".

### AdaBoost
Calcula as previsões de todos os previsores e os pondera usando os pesos dos previsores. Forma um novo previsor corrigindo o seu antecedor atribuindo mais peso para as instancias de treinamento classificadas erronhamente.

SAMME.R se baseia nas probabilidades da classe em vez de previsões, geralmente isso apresenta um resultado melhor.

In [12]:
from sklearn.ensemble import AdaBoostClassifier

ada_clf = AdaBoostClassifier(
    DecisionTreeClassifier(max_depth=1), n_estimators=200,
    algorithm="SAMME.R", learning_rate=0.5)
ada_clf.fit(X_train, y_train)

AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=1),
                   learning_rate=0.5, n_estimators=200)

### Gradient Boosting
Em vez de ajustar em relação os previsores das instancias a cada iteração, tenta ajustar o novo previsor em relação aos erros residuais feito pelo previsor anterior.

In [14]:
import numpy as np

np.random.seed(42)
X = np.random.rand(100, 1) - 0.5
y = 3*X[:, 0]**2 + 0.05 * np.random.randn(100)

from sklearn.ensemble import GradientBoostingRegressor

gbrt = GradientBoostingRegressor(max_depth=2, n_estimators=3, learning_rate=1.0)
gbrt.fit(X,y)

GradientBoostingRegressor(learning_rate=1.0, max_depth=2, n_estimators=3)

Encontrando o número ideal de árvores usnado o método staged_predict(), depois pega o melhor número de erro(menor) e treina o conjunto de dados com ele.

In [15]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

X_train, X_val, y_train, y_val = train_test_split(X, y)

gbrt = GradientBoostingRegressor(max_depth=2, n_estimators=120)
gbrt.fit(X_train, y_train)

errors = [mean_squared_error(y_val, y_pred)
         for y_pred in gbrt.staged_predict(X_val)]
bst_n_estimators = np.argmin(errors)

gbrt_best = GradientBoostingRegressor(max_depth=2, n_estimators=bst_n_estimators)
gbrt_best.fit(X_train,y_train)

GradientBoostingRegressor(max_depth=2, n_estimators=52)

Da para implementar uma parada antes de terminar tudo e depois ver qual o melhor.Por exemplo no código a seguir ele para após o erro não melhorar durante 5 iterações.

Obs: para fazer isso é preciso modificar o parametro warm_start para 'True'.

In [17]:
gbrt = GradientBoostingRegressor(max_depth=2, warm_start=True)

min_val_error = float("inf")
error_going_up = 0
for n_estimators in range(1, 120):
    gbrt.n_estimators = n_estimators
    gbrt.fit(X_train, y_train)
    y_pred = gbrt.predict(X_val)
    val_error = mean_squared_error(y_val,y_pred)
    if val_error < min_val_error:
        min_val_error = val_error
        error_going_up = 0
    else:
        error_going_up += 1
        if error_going_up == 5:
            break